# 9.2 Implement our own system assembling

In this tutorial we

* write integrators for $\int_T f v dx$ and $\int_T \nabla u \nabla v dx$: <br>
[myIntegrator.hpp](/edit/myIntegrator.hpp)
[myIntegrator.cpp](/edit/myIntegrator.cpp)


* put together element matrices to the global system matrix: <br>
[myAssembling.cpp](/edit/myAssembling.cpp)

In [ ]:
from ngsolve import *
import pyngcore   # for timers 

from ngsolve.webgui import Draw
from netgen.occ import unit_square

from ngsolve.fem import CompilePythonModule
from pathlib import Path

m = CompilePythonModule(Path('myassemblemodule.cpp'), init_function_name='mymodule')
dir (m)

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
fes = H1(mesh, order=1, dirichlet=".*")
u, v = fes.TnT()

### use our own integrators for element matrix calculation:

In [ ]:
a = BilinearForm(fes)
a += m.MyLaplace(1)
a.Assemble()

f = LinearForm(fes)
f += m.MySource(x)
f.Assemble();

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu);

### we can inspect the element matrix:

In [ ]:
mylap = m.MyLaplace(1)
ei = ElementId(VOL,17)
mylap.CalcElementMatrix(fes.GetFE(ei), mesh.GetTrafo(ei))

### use our own matrix assembling function:

In [ ]:
for l in range(4): mesh.Refine()
fes.Update()
gfu.Update()
f.Assemble();

In [ ]:
pyngcore.ResetTimers()
print ("num elements =", mesh.ne, ", ndof =", fes.ndof)
with TaskManager(pajetrace=10**8):
    # using our integrator
    mymatrix = m.MyAssembleMatrix(fes, m.MyLaplace(CF(1)), parallel=False)
    
    # using NGSolve built-in symbolic integrator
    # mymatrix = m.MyAssembleMatrix(fes, (grad(u)*grad(v)*dx)[0].MakeBFI(), parallel=True)

# print ("my matrix = ", mymat)

if fes.ndof < 100000:
    gfu.vec.data = mymatrix.Inverse(fes.FreeDofs()) * f.vec
    Draw (gfu);

### Compare timings for 

  * `MyLaplace` and `grad(u)*grad(v)*dx`
  * sequential and parallel
  * for different polynomial orders

In [ ]:
for t in pyngcore.Timers(): 
    if t["name"].startswith("MyAssemble"): 
        print (t["name"],"   ", t["time"])

### Exercise:


   * Implement `MyAssembleVector` for building the right hand side vector for  $\int_\Omega f v \, dx$
   * Implement `MyNeumannIntegrator` for evaluating $\int_{\partial \Omega} g v \, ds$